In [72]:
#Prediction model using LightGBM by Microsoft

In [73]:
import numpy as np

In [74]:
import matplotlib.pyplot as plt

In [75]:
import pandas as pd

In [76]:
from sklearn.preprocessing import StandardScaler

In [77]:
import lightgbm as lgb

In [78]:
from sklearn.model_selection import train_test_split

In [79]:
from sklearn.preprocessing import LabelBinarizer

In [80]:
from sklearn import datasets

In [81]:
train = pd.read_csv(r"C:\Users\comic\NFL-Predictor\Data\Train-Data\Compiled\testNOPERCENTAGES.csv")#Data we will train with preview
train.head()

,T1,T1 Outcome,T1 Score,T1 Location,T1 Avg Points,T1 Avg Yards,T1 NPY/A,T1 NRY/A,T1 3rd,T1 4th,...,T2 NPY/A,T2 NRY/A,T2 3rd,T2 4th,T2 Avg Points Def,T2 Avg Yards Def,T2 NPY/A Def,T2 NRY/A Def,T2 3rd Def,T2 4th Def
0,San Francisco 49ers,1,16,Away,22.9375,356.3125,5.8,4.6,0.52,0.62,...,6.9,3.9,0.39,0.36,17.4375,309.3125,5.4,4.5,0.37,0.38
1,Atlanta Falcons,0,34,Away,25.1250,345.9375,6.1,4.5,0.42,0.60,...,6.0,4.3,0.39,0.54,20.5000,311.5625,5.2,4.8,0.34,0.38
2,Buffalo Bills,0,31,Home,23.6875,349.4375,6.0,4.1,0.43,0.58,...,6.6,4.0,0.40,0.33,21.0000,341.4375,6.0,4.6,0.44,0.56
3,San Diego Chargers,1,34,Away,20.8125,332.8125,5.7,4.6,0.37,0.33,...,5.5,4.1,0.39,0.37,28.5000,329.0625,6.8,4.0,0.42,0.50
4,Miami Dolphins,1,49,Home,23.6250,337.0000,6.0,4.7,0.38,0.30,...,5.0,4.1,0.30,0.40,28.1875,382.3125,6.7,4.0,0.45,0.73


In [82]:
test = pd.read_csv(r"C:\Users\comic\NFL-Predictor\Data\Test-Data\Matchups\Matchup2019.csv")#Data eventually tested with preview
test.head()

,T1,T1 Outcome,T1 Score,T1 Location,T1 Avg Points,T1 Avg Yards,T1 NPY/A,T1 NRY/A,T1 3rd,T1 4th,...,T2 NPY/A,T2 NRY/A,T2 3rd,T2 4th,T2 Avg Points Def,T2 Avg Yards Def,T2 NPY/A Def,T2 NRY/A Def,T2 3rd Def,T2 4th Def
0,Chicago Bears,0,3,Home,17.5000,296.8125,5.3,3.7,0.356,0.333,...,6.1,4.4,0.360,0.500,19.5625,352.6250,6.3,4.7,0.381,0.444
1,Tennessee Titans,1,43,Away,25.1250,362.8125,7.1,5.0,0.378,0.286,...,6.1,4.8,0.362,0.412,24.5625,361.5625,6.3,5.0,0.379,0.353
2,Minnesota Vikings,1,28,Home,25.4375,353.5000,7.1,4.5,0.428,0.533,...,6.4,3.8,0.420,0.619,24.9375,355.8125,6.9,4.2,0.399,0.600
3,Washington Redskins,0,27,Away,16.6250,274.6875,5.3,4.4,0.291,0.429,...,5.9,4.3,0.454,0.333,22.1250,331.6875,6.3,4.1,0.341,0.611
4,Los Angeles Rams,1,30,Away,24.6250,374.8750,6.9,3.7,0.399,0.231,...,5.3,4.7,0.319,0.500,29.3750,374.5000,6.2,5.2,0.399,0.667


In [83]:
train = train.drop(["T1", "T1 Score", "T2", "T2 Score", "T2 Outcome"], axis =1)
#Remove columns don't want considered. 

In [84]:
print(train)#See what we have remaining

      T1 Outcome T1 Location  T1 Avg Points  T1 Avg Yards  T1 NPY/A  T1 NRY/A  \
0              1        Away        22.9375      356.3125       5.8       4.6   
1              0        Away        25.1250      345.9375       6.1       4.5   
2              0        Home        23.6875      349.4375       6.0       4.1   
3              1        Away        20.8125      332.8125       5.7       4.6   
4              1        Home        23.6250      337.0000       6.0       4.7   
...          ...         ...            ...           ...       ...       ...   
4347           1        Home        24.3125      374.9375       6.1       4.5   
4348           0        Away        18.1250      336.1875       6.2       4.2   
4349           0        Home        17.5625      299.6875       5.5       4.3   
4350           1        Away        26.7500      372.6250       7.5       4.7   
4351           0        Home        19.3750      312.6250       6.1       4.4   

      T1 3rd  T1 4th  T1 Av

In [85]:
#Turn Home into a 1 value and Away into a 0 value to get data into good format for LightGBM
train=train.replace(to_replace="Home",value="1")
train=train.replace(to_replace="Away",value="0")


In [86]:
print(train)
#See how it updated 

      T1 Outcome T1 Location  T1 Avg Points  T1 Avg Yards  T1 NPY/A  T1 NRY/A  \
0              1           0        22.9375      356.3125       5.8       4.6   
1              0           0        25.1250      345.9375       6.1       4.5   
2              0           1        23.6875      349.4375       6.0       4.1   
3              1           0        20.8125      332.8125       5.7       4.6   
4              1           1        23.6250      337.0000       6.0       4.7   
...          ...         ...            ...           ...       ...       ...   
4347           1           1        24.3125      374.9375       6.1       4.5   
4348           0           0        18.1250      336.1875       6.2       4.2   
4349           0           1        17.5625      299.6875       5.5       4.3   
4350           1           0        26.7500      372.6250       7.5       4.7   
4351           0           1        19.3750      312.6250       6.1       4.4   

      T1 3rd  T1 4th  T1 Av

In [87]:
train['y'] = 1
y_label = train['y']
del train['y']

In [88]:
categorical_features = [col for col in train.columns if train[col].dtype == 'object']
one_hot_data = pd.get_dummies(train, columns=categorical_features)#One hot encoding

In [89]:
train_x = one_hot_data.iloc[:-380,:]
test_x = one_hot_data.iloc[-380:,:]
train_y = y_label.iloc[:-380]
test_y = y_label.iloc[-380:]
#split data needed for GBM 

In [90]:
train_lgb_x, valid_lgb_x, train_lgb_y, valid_lgb_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True)

In [91]:
train_data=lgb.Dataset(train_lgb_x,label=train_lgb_y)
valid_data=lgb.Dataset(valid_lgb_x,label=valid_lgb_y)

In [111]:
#Need to mess with parameters and figure out what will fully train

params = {'boosting_type': 'gbdt',
          'max_depth' : 10,
          'objective': 'multiclass',
          'nthread': 5,
          'num_leaves': 10,
          'learning_rate': 0.05,
          'max_bin': 512,
          #'min_data_in_leaf': 5,
          #'feature_pre_filter': False,
          
          
          #'min_data': 1,
          #'subsample_for_bin': 200,
          #'subsample': 1,
          #'subsample_freq': 1,
          #'colsample_bytree': 0.8,
          #'reg_alpha': 5,
          'reg_lambda': 1.4,
          #'min_split_gain': 0.5,
          #'min_child_weight': 1,
          #'min_child_samples': 5,
          #'scale_pos_weight': 1,
          'num_class' : 3,
          'metric' : 'multi_logloss'
          }
#Change parameters 


#Started with these 
"""
params = {'boosting_type': 'gbdt',
          #'max_depth' : 10,
          'objective': 'multiclass',
          'nthread': 5,
          'num_leaves': 32,
          'learning_rate': 0.05,
          'max_bin': 512,
          #'subsample_for_bin': 200,
          #'subsample': 1,
          #'subsample_freq': 1,
          #'colsample_bytree': 0.8,
          #'reg_alpha': 5,
          'reg_lambda': 1.4,
          #'min_split_gain': 0.5,
          #'min_child_weight': 1,
          #'min_child_samples': 5,
          #'scale_pos_weight': 1,
          'num_class' : 3,
          'metric' : 'multi_logloss'
          }"""

"\nparams = {'boosting_type': 'gbdt',\n          #'max_depth' : 10,\n          'objective': 'multiclass',\n          'nthread': 5,\n          'num_leaves': 32,\n          'learning_rate': 0.05,\n          'max_bin': 512,\n          #'subsample_for_bin': 200,\n          #'subsample': 1,\n          #'subsample_freq': 1,\n          #'colsample_bytree': 0.8,\n          #'reg_alpha': 5,\n          'reg_lambda': 1.4,\n          #'min_split_gain': 0.5,\n          #'min_child_weight': 1,\n          #'min_child_samples': 5,\n          #'scale_pos_weight': 1,\n          'num_class' : 3,\n          'metric' : 'multi_logloss'\n          }"

In [112]:
#Actual training
lgbm = lgb.train(params,
                 train_data,
                 300,
                 #valid_sets=valid_data,
                 #min_data = 1,
                 verbose_eval= 10
                 )

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002574 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3280
[LightGBM] [Info] Number of data points in the train set: 3177, number of used features: 29
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee